In [83]:
pip install scikit_posthocs

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-py3-none-any.whl size=37902 sha256=fcd905a1ab138e451fd294b0d26f3dbc47da7751f597501843cd93fa120a55a0
  Stored in directory: c:\users\mrgab\appdata\local\pip\cache\wheels\59\a1\76\8954a8efb2e143534be5943de94a245c673b25b9ce71c3aab3
Successfully built scikit-posthocs
Note: you may need to restart the kernel to use updated packages.


In [106]:
import pandas as pd
from scipy import stats
import numpy as np
import scikit_posthocs as sp

In [107]:
mann_kendall_results = pd.read_csv("mann_kendall_results.csv")

In [121]:
metrics = mann_kendall_results.columns.values.tolist()
metrics.remove("full_name")
metrics.remove("framework")
metrics.remove("questions pvalue")
for metric in metrics:
    if "trend" in metric:
        metrics.remove(metric)

In [120]:
for metric in metrics:
    print(f"A metrica {metric} tem distribuição normal? ", stats.shapiro(mann_kendall_results[metric]).pvalue > 0.05)

A metrica JAVA pvalue tem distribuição normal?  False
A metrica CLASSES pvalue tem distribuição normal?  False
A metrica PKG pvalue tem distribuição normal?  False
A metrica BWR pvalue tem distribuição normal?  False
A metrica SAR pvalue tem distribuição normal?  False
A metrica FANIN pvalue tem distribuição normal?  False
A metrica FANOUT pvalue tem distribuição normal?  False
A metrica CC pvalue tem distribuição normal?  False
A metrica CCM pvalue tem distribuição normal?  False
A metrica CCS pvalue tem distribuição normal?  False
A metrica EC pvalue tem distribuição normal?  False
A metrica KNTS pvalue tem distribuição normal?  False
A metrica NPATH pvalue tem distribuição normal?  False
A metrica NPLOG pvalue tem distribuição normal?  False
A metrica PL pvalue tem distribuição normal?  False
A metrica LOC pvalue tem distribuição normal?  False
A metrica DLOC pvalue tem distribuição normal?  False
A metrica ELOC pvalue tem distribuição normal?  False
A metrica CLOC pvalue tem distri

In [123]:
increasing = dict()
decreasing = dict()
no_trend = dict()

for metric in metrics:
    metric = metric.split(" ")[0]
    increasing[metric] = list() 
    decreasing[metric] = list()
    no_trend[metric]   = list()

for index in range(0, len(mann_kendall_results)):
    for metric in metrics:
        metric = metric.split(" ")[0]
        value = mann_kendall_results.loc[index, metric+" trend"]
        if (value == "increasing"):
            increasing[metric].append(mann_kendall_results.loc[index, metric+" pvalue"])
        elif (value == "decreasing"):
            decreasing[metric].append(mann_kendall_results.loc[index, metric+" pvalue"])
        else:
            no_trend[metric].append(mann_kendall_results.loc[index, metric+" pvalue"])

In [124]:
for metric in metrics:
    metric = metric.split(" ")[0]
    kruskal = stats.kruskal(increasing[metric], decreasing[metric], no_trend[metric], nan_policy="raise")
    print(f"Are the results of {metric} means that the three groups have difference statistically? {kruskal.pvalue <= 0.05}")

Are the results of JAVA means that the three groups have difference statistically? True
Are the results of CLASSES means that the three groups have difference statistically? True
Are the results of PKG means that the three groups have difference statistically? True
Are the results of BWR means that the three groups have difference statistically? True
Are the results of SAR means that the three groups have difference statistically? True
Are the results of FANIN means that the three groups have difference statistically? True
Are the results of FANOUT means that the three groups have difference statistically? True
Are the results of CC means that the three groups have difference statistically? True
Are the results of CCM means that the three groups have difference statistically? True
Are the results of CCS means that the three groups have difference statistically? True
Are the results of EC means that the three groups have difference statistically? True
Are the results of KNTS means that 

In [125]:
from __future__ import division


def cliffsDelta(lst1, lst2, **dull):

    """Returns delta and true if there are more than 'dull' differences"""
    if not dull:
        dull = {'small': 0.147, 'medium': 0.33, 'large': 0.474} # effect sizes from (Hess and Kromrey, 2004)
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats, x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] < x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
    d = (more - less) / (m*n)
    size = lookup_size(d, dull)
    return d, size


def lookup_size(delta: float, dull: dict) -> str:
    """
    :type delta: float
    :type dull: dict, a dictionary of small, medium, large thresholds.
    """
    delta = abs(delta)
    if delta < dull['small']:
        return 'negligible'
    if dull['small'] <= delta < dull['medium']:
        return 'small'
    if dull['medium'] <= delta < dull['large']:
        return 'medium'
    if delta >= dull['large']:
        return 'large'


def runs(lst):
    """Iterator, chunks repeated values"""
    for j, two in enumerate(lst):
        if j == 0:
            one, i = two, 0
        if one != two:
            yield j - i, one
            i = j
        one = two
    yield j - i + 1, two

In [157]:
metric = "NPATH"

In [158]:
cliffsDelta(increasing[metric], decreasing[metric])

(0.3046275160821747, 'small')

In [159]:
cliffsDelta(decreasing[metric], increasing[metric])

(-0.3046275160821747, 'small')

In [160]:
cliffsDelta(increasing[metric], no_trend[metric])

(-1.0, 'large')

In [161]:
cliffsDelta(decreasing[metric], no_trend[metric])

(-1.0, 'large')

In [131]:
for metric in metrics:
    metric = metric.split(" ")[0]
    print(sp.posthoc_dunn([increasing[metric], decreasing[metric], no_trend[metric]]).sum(axis=1))

1    1.043855
2    1.043855
3    1.000000
dtype: float64
1    1.171314
2    1.171314
3    1.000000
dtype: float64
1    1.181702
2    1.181702
3    1.000000
dtype: float64
1    1.823552
2    1.823552
3    1.000000
dtype: float64
1    1.02774
2    1.02774
3    1.00000
dtype: float64
1    1.094547
2    1.094547
3    1.000000
dtype: float64
1    1.516221
2    1.516221
3    1.000000
dtype: float64
1    1.013183
2    1.013183
3    1.000000
dtype: float64
1    1.018899
2    1.018899
3    1.000000
dtype: float64
1    1.01205
2    1.01205
3    1.00000
dtype: float64
1    1.095847
2    1.095847
3    1.000000
dtype: float64
1    1.860947
2    1.860947
3    1.000000
dtype: float64
1    1.020428
2    1.020428
3    1.000000
dtype: float64
1    1.792671
2    1.792671
3    1.000000
dtype: float64
1    1.042429
2    1.042429
3    1.000000
dtype: float64
1    1.036799
2    1.036799
3    1.000000
dtype: float64
1    1.48505
2    1.48505
3    1.00000
dtype: float64
1    1.182872
2    1.182872
3    1.00000